<a href="https://colab.research.google.com/github/Frynnn-69/mini-research-bdc2025/blob/main/01_data_exploration/02_data_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =================================================================
# Warming up + Setup
# =================================================================
from google.colab import drive
import os

drive.mount('/content/drive')

repo_path = '/content/drive/My Drive/mini-research-bdc2025/'
os.chdir(repo_path)
!git pull


# --- UBAH PATH INI SESUAI LOKASI NOTEBOOK ---
notebook_folder = '01_data_exploration/'

working_dir = os.path.join(repo_path, notebook_folder)
os.chdir(working_dir)

print(f"\n✅ Setup Selesai. Anda sekarang bekerja di folder:")
!pwd

Mounted at /content/drive
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 22 (delta 8), reused 20 (delta 6), pack-reused 0 (from 0)
Unpacking objects: 100% (22/22), 17.46 KiB | 5.00 KiB/s, done.
From https://github.com/frynnn-69/mini-research-bdc2025
   375d8e8..3b7a1c0  main       -> origin/main
Updating 315bf44..3b7a1c0
Fast-forward
 01_data_exploration/02_final_data_validation.py |  61 ++
 01_data_exploration/datatrain.csv               |   5 +-
 01_data_exploration/datatrain_cleaned.csv       |   3 +-
 01_data_exploration/datatrain_final.csv         | 757 ++++++++++++++++++++++++
 01_data_exploration/qualitative_notes.md        | 205 +++++++
 02_literature_review/external_datasets.md       |  70 +++
 02_literature_review/pretrained_models.md       |  37 ++
 notes.md                                        |  66 +++
 8 files changed, 1201 insertions(+), 3 deletions(-)
 create mode 1006

In [3]:
import pandas as pd

!pip install -q yt-dlp

csv_path = 'datatrain_final.csv'

try:
    df = pd.read_csv(csv_path)
    print(f"\nBerhasil memuat '{csv_path}'!")
    print(f"Jumlah video yang akan diunduh: {len(df)}")
    display(df.head())
except FileNotFoundError:
    print(f"Error: File '{csv_path}' tidak ditemukan.")


Berhasil memuat 'datatrain_final.csv'!
Jumlah video yang akan diunduh: 755


,id,video,emotion
0,1,https://www.instagram.com/reel/DNKcHgdA-d1/?ig...,Surprise
1,2,https://www.instagram.com/reel/DNHwrh2gnBm/?ig...,Surprise
2,3,https://www.instagram.com/reel/DM7QsjnRCoa/?ig...,Surprise
3,4,https://www.instagram.com/reel/DNBBEt6Paxj/?ig...,Surprise
4,5,https://www.instagram.com/reel/DMz13fQzZsN/?ig...,Proud


In [7]:
import os
import subprocess
import time
import random

cookies_file_path = '/content/cookies.txt'
if not os.path.exists(cookies_file_path):
    print("File 'cookies.txt' not found!")
else:
    print("File 'cookies.txt' was found!")

    base_storage_path = '/content/drive/My Drive/BDC_DATASET/'
    video_train_path = os.path.join(base_storage_path, 'videos', 'train')
    os.makedirs(video_train_path, exist_ok=True)
    print(f"The video will be saved in: {video_train_path}")
    print("-" * 30)

    for index, row in df.iterrows():
        video_id = row['id']
        video_url = row['video']

        output_template = os.path.join(video_train_path, f"video_{video_id}.%(ext)s")

        if any(fname.startswith(f"video_{video_id}.") for fname in os.listdir(video_train_path)):
            continue

        print(f"Trying to download Video ID {video_id}...")

        command = f"yt-dlp -q --cookies '{cookies_file_path}' -f 'bv*[ext=mp4]+ba[ext=m4a]/b[ext=mp4] / bv*+ba/b' -o '{output_template}' '{video_url}'"

        try:
            subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
            print(f"Video ID {video_id}: Downloaded successfully.")
        except subprocess.CalledProcessError as e:
            error_lines = e.stderr.strip().split('\n')
            print(f"Video ID {video_id}: Failed. Error: {error_lines[-1]}")

        sleep_time = random.uniform(1, 5)
        time.sleep(sleep_time)


    print("\nDone!")

File 'cookies.txt' was found!
The video will be saved in: /content/drive/My Drive/BDC_DATASET/videos/train
------------------------------
Trying to download Video ID 23...
Video ID 23: Downloaded successfully.
Trying to download Video ID 24...
Video ID 24: Downloaded successfully.
Trying to download Video ID 25...
Video ID 25: Downloaded successfully.
Trying to download Video ID 26...
Video ID 26: Downloaded successfully.
Trying to download Video ID 27...
Video ID 27: Downloaded successfully.
Trying to download Video ID 28...
Video ID 28: Downloaded successfully.
Trying to download Video ID 29...
Video ID 29: Downloaded successfully.
Trying to download Video ID 30...
Video ID 30: Downloaded successfully.
Trying to download Video ID 31...
Video ID 31: Downloaded successfully.
Trying to download Video ID 32...
Video ID 32: Downloaded successfully.
Trying to download Video ID 33...
Video ID 33: Downloaded successfully.
Trying to download Video ID 35...
Video ID 35: Downloaded successfully

In [8]:
# Extract Audio
from tqdm.auto import tqdm

video_train_path = '/content/drive/My Drive/BDC_DATASET/videos/train'
audio_train_path = '/content/drive/My Drive/BDC_DATASET/audio/train'
os.makedirs(audio_train_path, exist_ok=True)

print(f"The audio will be saved in: {audio_train_path}")
print("-" * 30)

video_files = os.listdir(video_train_path)

for video_filename in tqdm(video_files, desc="Extracting Audio"):
    source_video_path = os.path.join(video_train_path, video_filename)

    base_filename = os.path.splitext(video_filename)[0]
    output_audio_path = os.path.join(audio_train_path, f"{base_filename.replace('video_', 'audio_')}.wav")

    if os.path.exists(output_audio_path):
        continue

    command = f"ffmpeg -i '{source_video_path}' -vn -acodec pcm_s16le -ar 16000 -ac 1 '{output_audio_path}'"

    try:
        subprocess.run(command, shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except subprocess.CalledProcessError as e:
        print(f"Failed to extract audio from: {video_filename}")

print("\nAudio extraction process is complete!")

The audio will be saved in: /content/drive/My Drive/BDC_DATASET/audio/train
------------------------------


Extracting Audio:   0%|          | 0/752 [00:00<?, ?it/s]


Audio extraction process is complete!
